# All Models, All Features (Except Secondary Structure), One-hot Encoding

In [1]:
%load_ext watermark
%watermark -p pandas,numpy,scikit-learn,matplotlib --conda 

pandas      : 1.4.4
numpy       : 1.23.3
scikit-learn: 1.1.2
matplotlib  : 3.5.3

conda environment: hotspot



In [2]:
import pandas as pd
import numpy as np

# 1) Data Preparation

## Binarize

In [3]:
df_train = pd.read_csv('../dataset/TrainingDataset.csv')
df_test = pd.read_csv('../dataset/TestDataset.csv')

# 1 & 2 -> 1
# 0 -> 0

binarized = df_train['3-class'].values.copy()
binarized[binarized == 2] = 1
df_train['2-class-merged-v1'] = binarized.astype(int)

binarized = df_test['3-class'].values.copy()
binarized[binarized == 2] = 1
df_test['2-class-merged-v1'] = binarized.astype(int)


y_train = df_train['2-class-merged-v1'].values
y_test = df_test['2-class-merged-v1'].values

## Use Sequence-Only Features

In [4]:
np.unique(df_train['residue'])

array(['C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q',
       'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object)

In [5]:
# convert aa char to int
codes = ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 
         'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
code_to_int = {c:i for i,c in enumerate(codes)}     
df_train['residue'] = df_train['residue'].map(code_to_int)
df_test['residue'] = df_test['residue'].map(code_to_int)

In [6]:
np.unique(df_train['secondary structure'])

array(['-', 'H', 'S', 'T'], dtype=object)

In [7]:
# convert secondary structure char to int
codes = ['H', 'S', 'T', '-']
code_to_int = {c:i for i,c in enumerate(codes)}     
df_train['secondary structure'] = df_train['secondary structure'].map(code_to_int)
df_test['secondary structure'] = df_test['secondary structure'].map(code_to_int)

feature_list = ['residue', 'consurf', 'secondary structure']

df_train = df_train[feature_list]
df_test = df_test[feature_list]

In [8]:
df_train['residue'] = df_train['residue'].astype('category')
df_test['residue'] = df_test['residue'].astype('category')

df_train['secondary structure'] = df_train['secondary structure'].astype('category')
df_test['secondary structure'] = df_test['secondary structure'].astype('category')

In [9]:
df_train.dtypes

residue                category
consurf                   int64
secondary structure    category
dtype: object

In [10]:
X_train = df_train[feature_list].values
X_test =  df_test[feature_list].values

In [11]:
X_train.shape

(732, 3)

In [12]:
y_train.shape

(732,)

In [13]:
X_test.shape

(314, 3)

In [14]:
y_test.shape

(314,)

## OneHot Encoding

In [15]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop='first')
ohe.fit(df_train[feature_list][['residue', 'secondary structure']])

OneHotEncoder(drop='first')

In [16]:
df_train_ohe = df_train.drop(columns=['residue', 'secondary structure'])
df_test_ohe = df_test.drop(columns=['residue', 'secondary structure'])

In [17]:
ohe_train = np.asarray(ohe.transform(df_train[feature_list][['residue', 'secondary structure']]).todense())
ohe_test = np.asarray(ohe.transform(df_test[feature_list][['residue', 'secondary structure']]).todense())

In [18]:
X_train_ohe = np.hstack((df_train_ohe.values, ohe_train))
X_test_ohe = np.hstack((df_test_ohe.values, ohe_test))

# 2) Hyperparameter Tuning & Evaluation

## Logistic Regression

### Baseline

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score



pipe = make_pipeline(StandardScaler(),
                     LogisticRegression(random_state=123))


score_names = ['precision', 'recall', 'f1']

for score_name in score_names:

    scores = cross_val_score(
        X=X_train_ohe,
        y=y_train,
        estimator=pipe,
        cv=10,
        scoring=score_name
    )
    
    print(f'{score_name}: {np.mean(scores):.2f} +/- {np.std(scores):.2f}')

precision: 0.54 +/- 0.05
recall: 0.45 +/- 0.10
f1: 0.49 +/- 0.07


### Tuned

In [20]:
import scipy.stats
import sklearn
from sklearn.model_selection import GridSearchCV


pipe = make_pipeline(StandardScaler(),
                     LogisticRegression(random_state=123, max_iter=1000))

params =  {
    'logisticregression__penalty':['l1', 'l2'],
    'logisticregression__C': [0.0001, 0.001, 0.01, 0.1, 0.0, 1.0, 10, 100, 1000],
}

gs = GridSearchCV(estimator=pipe, 
                  param_grid=params, 
                  scoring='f1', 
                  refit=True,
                  cv=10)

gs = gs.fit(X_train_ohe, y_train)
print('CV F1 score', gs.best_score_)
print('Best params', gs.best_params_)


score_names = ['precision', 'recall', 'f1']


for score_name in score_names:

    scores = cross_val_score(
        X=X_train_ohe,
        y=y_train,
        estimator=gs.best_estimator_,
        cv=10,
        n_jobs=-1,
        scoring=score_name
    )
    print(f'10-fold CV {score_name}: {np.mean(scores):.2f} +/- {np.std(scores):.2f}')

    
for score_name in score_names:
    
    scorer = sklearn.metrics.get_scorer(score_name)
    
    score = scorer(gs.best_estimator_, X_test_ohe, y_test)
    print(f'Test {score_name}: {score:.2f}')

CV F1 score 0.49331946128015014
Best params {'logisticregression__C': 10, 'logisticregression__penalty': 'l2'}


/Users/sebastianraschka/miniforge3/envs/hotspot/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
100 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/sebastianraschka/miniforge3/envs/hotspot/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/sebastianraschka/miniforge3/envs/hotspot/lib/python3.9/site-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/sebastianraschka/miniforge3/envs/hotspot/li

10-fold CV precision: 0.55 +/- 0.04
10-fold CV recall: 0.45 +/- 0.09
10-fold CV f1: 0.49 +/- 0.07
Test precision: 0.62
Test recall: 0.48
Test f1: 0.54


## Random Forest

### Baseline

In [21]:
from sklearn.ensemble import RandomForestClassifier


score_names = ['precision', 'recall', 'f1']

for score_name in score_names:

    scores = cross_val_score(
        X=X_train_ohe,
        y=y_train,
        estimator=RandomForestClassifier(random_state=123, n_estimators=1000),
        cv=10,
        n_jobs=-1,
        scoring=score_name
    )
    
    print(f'{score_name}: {np.mean(scores):.2f} +/- {np.std(scores):.2f}')

precision: 0.56 +/- 0.10
recall: 0.53 +/- 0.07
f1: 0.54 +/- 0.06


### Tuned

In [22]:
pipe = make_pipeline(RandomForestClassifier(random_state=123))

params =  {
    'randomforestclassifier__max_depth': [3, 5, 10],
    'randomforestclassifier__min_samples_split': [2, 5, 10],
    'randomforestclassifier__n_estimators': [1000],
}

gs = GridSearchCV(estimator=pipe, 
                  param_grid=params, 
                  scoring='f1', 
                  refit=True,
                  cv=10)

gs = gs.fit(X_train_ohe, y_train)
print('CV F1 score', gs.best_score_)
print('Best params', gs.best_params_)


score_names = ['precision', 'recall', 'f1']


for score_name in score_names:

    scores = cross_val_score(
        X=X_train_ohe,
        y=y_train,
        estimator=gs.best_estimator_,
        cv=10,
        n_jobs=-1,
        scoring=score_name
    )
    print(f'10-fold CV {score_name}: {np.mean(scores):.2f} +/- {np.std(scores):.2f}')

    
for score_name in score_names:
    
    scorer = sklearn.metrics.get_scorer(score_name)
    
    score = scorer(gs.best_estimator_, X_test_ohe, y_test)
    print(f'Test {score_name}: {score:.2f}')

CV F1 score 0.5352185095842519
Best params {'randomforestclassifier__max_depth': 10, 'randomforestclassifier__min_samples_split': 5, 'randomforestclassifier__n_estimators': 1000}
10-fold CV precision: 0.61 +/- 0.09
10-fold CV recall: 0.49 +/- 0.09
10-fold CV f1: 0.54 +/- 0.07
Test precision: 0.65
Test recall: 0.46
Test f1: 0.54


## HistGradientBoosting

In [23]:
from sklearn.ensemble import HistGradientBoostingClassifier


pipe = make_pipeline(HistGradientBoostingClassifier(random_state=123))

params = {
    'histgradientboostingclassifier__learning_rate': (0.01, 0.1, 1, 10),
    'histgradientboostingclassifier__max_leaf_nodes': (3, 10, 30)
}

gs = GridSearchCV(estimator=pipe, 
                  param_grid=params, 
                  scoring='f1', 
                  refit=True,
                  cv=10)

gs = gs.fit(X_train_ohe, y_train)
print('CV F1 score', gs.best_score_)
print('Best params', gs.best_params_)


score_names = ['precision', 'recall', 'f1']


for score_name in score_names:

    scores = cross_val_score(
        X=X_train_ohe,
        y=y_train,
        estimator=gs.best_estimator_,
        cv=10,
        n_jobs=-1,
        scoring=score_name
    )
    print(f'10-fold CV {score_name}: {np.mean(scores):.2f} +/- {np.std(scores):.2f}')

    
for score_name in score_names:
    
    scorer = sklearn.metrics.get_scorer(score_name)
    
    score = scorer(gs.best_estimator_, X_test_ohe, y_test)
    print(f'Test {score_name}: {score:.2f}')

CV F1 score 0.5648596746225949
Best params {'histgradientboostingclassifier__learning_rate': 1, 'histgradientboostingclassifier__max_leaf_nodes': 30}
10-fold CV precision: 0.58 +/- 0.09
10-fold CV recall: 0.55 +/- 0.08
10-fold CV f1: 0.56 +/- 0.07
Test precision: 0.57
Test recall: 0.59
Test f1: 0.58


In [24]:
#from joblib import dump

#dump([ohe, gs.best_estimator_], 'gbm-seqonly.joblib') 

['gbm-seqonly.joblib']

In [25]:
#X_all_ohe = np.vstack((X_train_ohe, X_test_ohe))
#y_all_ohe = np.hstack((y_train, y_test))

#gs.best_estimator_.fit(X_all_ohe, y_all_ohe)

#dump([ohe, gs.best_estimator_], 'gbm-seqonly-hotspotter.joblib') 

['gbm-seqonly-hotspotter.joblib']

## Support Vector Machine

In [27]:
from sklearn.svm import SVC


pipe = make_pipeline(StandardScaler(), SVC(random_state=123))

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

params = [{'svc__C': param_range, 
               'svc__kernel': ['linear']},
              {'svc__C': param_range, 
               'svc__gamma': param_range, 
               'svc__kernel': ['rbf']}]

gs = GridSearchCV(estimator=pipe, 
                  param_grid=params, 
                  scoring='f1', 
                  refit=True,
                  cv=10)

gs = gs.fit(X_train_ohe, y_train)
print('CV F1 score', gs.best_score_)
print('Best params', gs.best_params_)


score_names = ['precision', 'recall', 'f1']


for score_name in score_names:

    scores = cross_val_score(
        X=X_train_ohe,
        y=y_train,
        estimator=gs.best_estimator_,
        cv=10,
        n_jobs=-1,
        scoring=score_name
    )
    print(f'10-fold CV {score_name}: {np.mean(scores):.2f} +/- {np.std(scores):.2f}')

    
for score_name in score_names:
    
    scorer = sklearn.metrics.get_scorer(score_name)
    
    score = scorer(gs.best_estimator_, X_test_ohe, y_test)
    print(f'Test {score_name}: {score:.2f}')

CV F1 score 0.5567712545157372
Best params {'svc__C': 100.0, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}
10-fold CV precision: 0.58 +/- 0.09
10-fold CV recall: 0.54 +/- 0.08
10-fold CV f1: 0.56 +/- 0.07
Test precision: 0.58
Test recall: 0.57
Test f1: 0.57


## Multilayer Perceptron

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import sklearn

from sklearn.neural_network import MLPClassifier


pipe = make_pipeline(StandardScaler(),
                     MLPClassifier(max_iter=10000, random_state=123))


params = {
    'mlpclassifier__hidden_layer_sizes': [(30, 20, 10), 
                                          (40, 20), 
                                          (20, 10), 
                                          (20,),
                                          (10,)],
    'mlpclassifier__activation': ['tanh', 'relu'],
    'mlpclassifier__solver': ['sgd', 'adam'],
    'mlpclassifier__alpha': [0.0001, 0.001, 0.05],
    'mlpclassifier__learning_rate': ['constant', 'adaptive'],
}

gs = GridSearchCV(estimator=pipe, 
                  param_grid=params, 
                  scoring='f1', 
                  refit=True,
                  n_jobs=-1,
                  cv=10)

gs = gs.fit(X_train_ohe, y_train)
print('CV F1 score', gs.best_score_)
print('Best params', gs.best_params_)


score_names = ['precision', 'recall', 'f1']


for score_name in score_names:

    scores = cross_val_score(
        X=X_train_ohe,
        y=y_train,
        estimator=gs.best_estimator_,
        cv=10,
        n_jobs=-1,
        scoring=score_name
    )
    print(f'10-fold CV {score_name}: {np.mean(scores):.2f} +/- {np.std(scores):.2f}')

    
for score_name in score_names:
    
    scorer = sklearn.metrics.get_scorer(score_name)
    
    score = scorer(gs.best_estimator_, X_test_ohe, y_test)
    print(f'Test {score_name}: {score:.2f}')

CV F1 score 0.554799788112198
Best params {'mlpclassifier__activation': 'tanh', 'mlpclassifier__alpha': 0.0001, 'mlpclassifier__hidden_layer_sizes': (20,), 'mlpclassifier__learning_rate': 'constant', 'mlpclassifier__solver': 'adam'}
10-fold CV precision: 0.56 +/- 0.08
10-fold CV recall: 0.56 +/- 0.10
10-fold CV f1: 0.55 +/- 0.07
Test precision: 0.58
Test recall: 0.57
Test f1: 0.57


In [20]:
from joblib import dump

dump([ohe, gs.best_estimator_], '../../hotspotter/mlp-seqfeatures.joblib') 

['../../hotspotter/mlp-seqfeatures.joblib']

In [21]:
X_all_ohe = np.vstack((X_train_ohe, X_test_ohe))
y_all_ohe = np.hstack((y_train, y_test))

gs.best_estimator_.fit(X_all_ohe, y_all_ohe)

dump([ohe, gs.best_estimator_], '../../hotspotter/mlp-seqfeatures-alldata.joblib') 

['../../hotspotter/mlp-seqfeatures-alldata.joblib']